In [1]:
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer , T5ForConditionalGeneration,Seq2SeqTrainer,Seq2SeqTrainingArguments
from transformers.models.t5.modeling_t5 import T5LayerSelfAttention , T5Attention

In [2]:
from TALib import TALib

In [3]:
ta_lib = TALib()

Map:   0%|          | 0/15159 [00:00<?, ? examples/s]

Map:   0%|          | 0/3790 [00:00<?, ? examples/s]

In [4]:
# tokenizer = AutoTokenizer.from_pretrained(TALib.TK_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(TALib.CHECKPOINT)

In [5]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [6]:
TALib.show_param_ratio(model)

1.0

In [7]:
trainer = ta_lib.get_trainer(model=model , num_train_epochs=15 , batch_size=5 , output_dir="KY_billsum_model") 

/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [8]:
trainer.evaluate(ta_lib.tokenized_billsum_test)

/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'eval_loss': 1.49580979347229,
 'eval_rouge1': 0.241,
 'eval_rouge2': 0.1962,
 'eval_rougeL': 0.2333,
 'eval_rougeLsum': 0.2334,
 'eval_gen_len': 18.9997,
 'eval_runtime': 371.6962,
 'eval_samples_per_second': 8.795,
 'eval_steps_per_second': 1.76}

In [9]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.717000,1.461313,0.243500,0.197700,0.236200,0.236100,19.000000
2,1.628100,1.441524,0.243800,0.198400,0.236400,0.236300,18.999500
3,1.439500,1.427871,0.244200,0.199400,0.237000,0.236900,18.999500
4,1.567700,1.410927,0.244600,0.200100,0.237500,0.237400,18.999500
5,1.611700,1.403012,0.244800,0.200500,0.237600,0.237500,19.000000
6,1.665800,1.393306,0.245000,0.201000,0.238000,0.237900,19.000000
7,1.380400,1.387831,0.245400,0.201300,0.238400,0.238300,19.000000
8,1.493200,1.381272,0.245300,0.201200,0.238200,0.238200,19.000000
9,1.348400,1.376467,0.245200,0.201400,0.238200,0.238100,18.999500
10,1.462200,1.372788,0.245500,0.201900,0.238700,0.238600,19.000000


/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/root/code/python/NYCU-Data-Science-2024/.venv

TrainOutput(global_step=45480, training_loss=1.5199361290340374, metrics={'train_runtime': 18094.4938, 'train_samples_per_second': 12.567, 'train_steps_per_second': 2.513, 'total_flos': 6.154939105542144e+16, 'train_loss': 1.5199361290340374, 'epoch': 15.0})

In [10]:
trainer.evaluate(ta_lib.tokenized_billsum_test)

{'eval_loss': 1.3625977039337158,
 'eval_rouge1': 0.2448,
 'eval_rouge2': 0.2024,
 'eval_rougeL': 0.2379,
 'eval_rougeLsum': 0.2379,
 'eval_gen_len': 19.0,
 'eval_runtime': 380.2154,
 'eval_samples_per_second': 8.598,
 'eval_steps_per_second': 1.72,
 'epoch': 15.0}

In [11]:
import torch

import torch.nn.utils.prune as prune

In [12]:
from transformers.models.t5.modeling_t5 import T5LayerSelfAttention, T5LayerCrossAttention, T5LayerFF

In [13]:
parameters_to_prune = {"self_attention" : [] , "cross_attention":[] , "ffn":[] , "lm_head":[]}
for name, module in model.named_modules():
    # print(name , type(module))
    if isinstance(module ,T5LayerSelfAttention ):
        # print("SelfAttention " , module)
        
        for name_2 , item in module.named_modules():
            if isinstance(item , torch.nn.Linear):
                parameters_to_prune["self_attention"].append((item , "weight"))
                
    if isinstance(module ,T5LayerCrossAttention ):
        # print("CrossAttention " , module)
        
        for name_2 , item in module.named_modules():
            if isinstance(item , torch.nn.Linear):
                parameters_to_prune["cross_attention"].append((item , "weight"))
                
    if isinstance(module ,T5LayerFF ):
        # print("FFN " , module)
        
        for name_2 , item in module.named_modules():
            if isinstance(item , torch.nn.Linear):
                parameters_to_prune["ffn"].append((item , "weight"))
                
    if isinstance(module ,torch.nn.Linear ) and name == "lm_head":
        parameters_to_prune["lm_head"].append((module , "weight"))
        
    # if isinstance(module, torch.nn.Linear):
    #     parameters_to_prune.append((module, "weight"))


In [14]:
parameters_to_prune

{'self_attention': [(Linear(in_features=512, out_features=512, bias=False),
   'weight'),
  (Linear(in_features=512, out_features=512, bias=False), 'weight'),
  (Linear(in_features=512, out_features=512, bias=False), 'weight'),
  (Linear(in_features=512, out_features=512, bias=False), 'weight'),
  (Linear(in_features=512, out_features=512, bias=False), 'weight'),
  (Linear(in_features=512, out_features=512, bias=False), 'weight'),
  (Linear(in_features=512, out_features=512, bias=False), 'weight'),
  (Linear(in_features=512, out_features=512, bias=False), 'weight'),
  (Linear(in_features=512, out_features=512, bias=False), 'weight'),
  (Linear(in_features=512, out_features=512, bias=False), 'weight'),
  (Linear(in_features=512, out_features=512, bias=False), 'weight'),
  (Linear(in_features=512, out_features=512, bias=False), 'weight'),
  (Linear(in_features=512, out_features=512, bias=False), 'weight'),
  (Linear(in_features=512, out_features=512, bias=False), 'weight'),
  (Linear(in_

In [15]:
self_attention_prune_amount = 0.9
cross_attention_prune_amount = 0.4
ff_prune_amount = 0.7
lm_head_amount = 0.8

In [16]:
trainer = ta_lib.get_trainer(model=model )

/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [16]:
def prune_model():
    prune.global_unstructured(
        parameters_to_prune["self_attention"],
        pruning_method=prune.L1Unstructured,
        amount=self_attention_prune_amount,
    )
    prune.global_unstructured(
        parameters_to_prune["cross_attention"],
        pruning_method=prune.L1Unstructured,
        amount=cross_attention_prune_amount,
    )
    prune.global_unstructured(
        parameters_to_prune["ffn"],
        pruning_method=prune.L1Unstructured,
        amount=ff_prune_amount,
    )
    prune.global_unstructured(
        parameters_to_prune["lm_head"],
        pruning_method=prune.L1Unstructured,
        amount=lm_head_amount,
    )
    return 

In [13]:
# prune_model()

In [17]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [18]:
TALib.show_param_ratio(model)

1.0

In [19]:
prune.global_unstructured(
    parameters_to_prune["self_attention"],
    pruning_method=prune.L1Unstructured,
    amount=self_attention_prune_amount,
)

In [20]:
TALib.show_param_ratio(model)

0.8128366470336914

In [21]:
trainer = ta_lib.get_trainer(model)

In [22]:
trainer.evaluate(ta_lib.tokenized_billsum_test)

{'eval_loss': 3.7154033184051514,
 'eval_rouge1': 0.039,
 'eval_rouge2': 0.0055,
 'eval_rougeL': 0.0343,
 'eval_rougeLsum': 0.0343,
 'eval_gen_len': 18.9376,
 'eval_runtime': 777.4345,
 'eval_samples_per_second': 4.205,
 'eval_steps_per_second': 2.103}

In [23]:
trainer = ta_lib.get_trainer(model , num_train_epochs=15 , batch_size=5 ,output_dir="./output/pruning_v3")

In [24]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.552600,2.214810,0.227400,0.174500,0.216400,0.216400,18.970400
2,2.325900,2.081054,0.233000,0.183300,0.224000,0.223900,18.988400
3,2.088200,2.019241,0.235000,0.186400,0.226600,0.226600,18.991800
4,2.173100,1.966064,0.236500,0.188700,0.228300,0.228500,18.987900
5,2.201100,1.934946,0.237800,0.190200,0.229700,0.229700,18.992600
6,2.240900,1.904227,0.238900,0.191500,0.230800,0.230900,18.996600
7,2.009100,1.887683,0.239600,0.192500,0.231600,0.231600,18.996000
8,2.062400,1.872282,0.239500,0.192700,0.231700,0.231700,18.996000
9,1.905300,1.858434,0.240100,0.193500,0.232400,0.232400,18.997900
10,2.008100,1.851333,0.240200,0.193800,0.232500,0.232600,18.998200


/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/root/code/python/NYCU-Data-Science-2024/.venv

TrainOutput(global_step=45480, training_loss=2.1489335898044555, metrics={'train_runtime': 18728.5176, 'train_samples_per_second': 12.141, 'train_steps_per_second': 2.428, 'total_flos': 6.154939105542144e+16, 'train_loss': 2.1489335898044555, 'epoch': 15.0})

In [25]:
trainer.evaluate(ta_lib.tokenized_billsum_test)

{'eval_loss': 1.8115092515945435,
 'eval_rouge1': 0.2391,
 'eval_rouge2': 0.1938,
 'eval_rougeL': 0.2311,
 'eval_rougeLsum': 0.2311,
 'eval_gen_len': 18.9969,
 'eval_runtime': 372.1457,
 'eval_samples_per_second': 8.784,
 'eval_steps_per_second': 1.757,
 'epoch': 15.0}

In [26]:
TALib.show_param_ratio(model)

0.8128366470336914

In [27]:
prune.global_unstructured(
    parameters_to_prune["cross_attention"],
    pruning_method=prune.L1Unstructured,
    amount=cross_attention_prune_amount,
)

In [28]:
TALib.show_param_ratio(model)

0.7712448239326477

In [29]:
trainer = ta_lib.get_trainer(model , num_train_epochs=20 , batch_size=5 ,output_dir="./output/pruning_v3")

/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [30]:
trainer.evaluate(ta_lib.tokenized_billsum_test)

{'eval_loss': 6.325265407562256,
 'eval_rouge1': 0.0139,
 'eval_rouge2': 0.0001,
 'eval_rougeL': 0.0137,
 'eval_rougeLsum': 0.0137,
 'eval_gen_len': 18.4628,
 'eval_runtime': 386.4415,
 'eval_samples_per_second': 8.459,
 'eval_steps_per_second': 1.692}

In [31]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.937200,2.497152,0.215000,0.122800,0.187100,0.187200,18.906100
2,2.626800,2.276995,0.223700,0.146500,0.201000,0.201000,18.954100
3,2.359500,2.180384,0.229900,0.176400,0.218700,0.218700,18.976500
4,2.424400,2.134937,0.231800,0.181000,0.222300,0.222300,18.982800
5,2.440600,2.099964,0.233500,0.183400,0.224100,0.224100,18.991600
6,2.463400,2.067773,0.234700,0.185000,0.225500,0.225500,18.992600
7,2.249200,2.049789,0.235200,0.186100,0.226300,0.226300,18.995800
8,2.272700,2.036448,0.236000,0.186800,0.226900,0.227000,18.996600
9,2.139800,2.015613,0.236200,0.187400,0.227400,0.227500,18.997100
10,2.201200,2.005004,0.236700,0.188100,0.228000,0.228100,18.994700


/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/root/code/python/NYCU-Data-Science-2024/.venv

TrainOutput(global_step=60640, training_loss=2.3469779018990913, metrics={'train_runtime': 24651.7094, 'train_samples_per_second': 12.299, 'train_steps_per_second': 2.46, 'total_flos': 8.206585474056192e+16, 'train_loss': 2.3469779018990913, 'epoch': 20.0})

In [32]:
trainer.evaluate(ta_lib.tokenized_billsum_test)

{'eval_loss': 1.9339864253997803,
 'eval_rouge1': 0.238,
 'eval_rouge2': 0.1907,
 'eval_rougeL': 0.2293,
 'eval_rougeLsum': 0.2294,
 'eval_gen_len': 18.9939,
 'eval_runtime': 380.9624,
 'eval_samples_per_second': 8.581,
 'eval_steps_per_second': 1.717,
 'epoch': 20.0}

In [33]:
prune.global_unstructured(
    parameters_to_prune["ffn"],
    pruning_method=prune.L1Unstructured,
    amount=ff_prune_amount,
)

In [34]:
TALib.show_param_ratio(model)

0.48010188341140747

In [35]:
trainer = ta_lib.get_trainer(model , num_train_epochs=15 , batch_size=5 ,output_dir="./output/pruning_v3")

/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [36]:
trainer.evaluate(ta_lib.tokenized_billsum_test)

{'eval_loss': 6.28693962097168,
 'eval_rouge1': 0.0146,
 'eval_rouge2': 0.0001,
 'eval_rougeL': 0.0146,
 'eval_rougeLsum': 0.0146,
 'eval_gen_len': 19.0,
 'eval_runtime': 392.3616,
 'eval_samples_per_second': 8.332,
 'eval_steps_per_second': 1.667}

In [37]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,3.086800,2.531628,0.221000,0.147900,0.200400,0.200400,18.948500
2,2.843200,2.433920,0.227800,0.171700,0.216000,0.216000,18.990800
3,2.588100,2.386825,0.228000,0.177100,0.218600,0.218600,18.990000
4,2.689200,2.349193,0.229700,0.179400,0.220500,0.220500,18.993400
5,2.732400,2.324470,0.230800,0.180900,0.221900,0.221800,18.995800
6,2.734000,2.302664,0.231300,0.181500,0.222500,0.222500,18.987100
7,2.572900,2.287221,0.232300,0.182500,0.223500,0.223500,18.987600
8,2.583400,2.277168,0.232500,0.182600,0.223700,0.223700,18.989700
9,2.410300,2.261664,0.232800,0.183100,0.224200,0.224100,18.992300
10,2.518400,2.256471,0.233400,0.183700,0.224700,0.224700,18.998200


/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/root/code/python/NYCU-Data-Science-2024/.venv

TrainOutput(global_step=45480, training_loss=2.6936071073369385, metrics={'train_runtime': 19609.6108, 'train_samples_per_second': 11.596, 'train_steps_per_second': 2.319, 'total_flos': 6.154939105542144e+16, 'train_loss': 2.6936071073369385, 'epoch': 15.0})

In [38]:
trainer.evaluate(ta_lib.tokenized_billsum_test)

{'eval_loss': 2.212543249130249,
 'eval_rouge1': 0.2343,
 'eval_rouge2': 0.1853,
 'eval_rougeL': 0.2254,
 'eval_rougeLsum': 0.2253,
 'eval_gen_len': 18.9951,
 'eval_runtime': 400.8947,
 'eval_samples_per_second': 8.154,
 'eval_steps_per_second': 1.631,
 'epoch': 15.0}

In [39]:
prune.global_unstructured(
    parameters_to_prune["lm_head"],
    pruning_method=prune.L1Unstructured,
    amount=lm_head_amount,
)

In [40]:
TALib.show_param_ratio(model)

0.26261115074157715

In [41]:
trainer = ta_lib.get_trainer(model , num_train_epochs=30 , batch_size=5 ,output_dir="./output/pruning_v3")

/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [42]:
trainer.evaluate(ta_lib.tokenized_billsum_test)

{'eval_loss': 6.514431476593018,
 'eval_rouge1': 0.051,
 'eval_rouge2': 0.0224,
 'eval_rougeL': 0.0493,
 'eval_rougeLsum': 0.0493,
 'eval_gen_len': 18.6164,
 'eval_runtime': 388.3105,
 'eval_samples_per_second': 8.419,
 'eval_steps_per_second': 1.684}

In [43]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,5.002100,4.273895,0.122600,0.068500,0.114900,0.114900,18.964900
2,4.291200,3.746312,0.196400,0.131100,0.184800,0.184800,18.992900
3,3.858700,3.498810,0.208600,0.149900,0.198500,0.198500,18.959100
4,3.912800,3.349042,0.211100,0.155100,0.201600,0.201600,18.935400
5,3.908200,3.247055,0.211700,0.158000,0.203100,0.203200,18.950700
6,3.855600,3.164701,0.214400,0.162300,0.206200,0.206200,18.958600
7,3.588700,3.108447,0.217700,0.165600,0.209200,0.209200,18.967300
8,3.578100,3.059389,0.221000,0.168900,0.212300,0.212400,18.971500
9,3.387000,3.010522,0.223500,0.171500,0.214700,0.214800,18.975700
10,3.496100,2.977571,0.225500,0.173300,0.216600,0.216700,18.973900


/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/root/code/python/NYCU-Data-Science-2024/.venv

TrainOutput(global_step=90960, training_loss=3.5635117952091817, metrics={'train_runtime': 40046.5942, 'train_samples_per_second': 11.356, 'train_steps_per_second': 2.271, 'total_flos': 1.2309878211084288e+17, 'train_loss': 3.5635117952091817, 'epoch': 30.0})

In [44]:
trainer.evaluate(ta_lib.tokenized_billsum_test)

{'eval_loss': 2.7338263988494873,
 'eval_rouge1': 0.2343,
 'eval_rouge2': 0.1838,
 'eval_rougeL': 0.2253,
 'eval_rougeLsum': 0.2253,
 'eval_gen_len': 18.9838,
 'eval_runtime': 389.4141,
 'eval_samples_per_second': 8.395,
 'eval_steps_per_second': 1.679,
 'epoch': 30.0}

In [45]:
TALib.show_param_ratio(model)

0.26261115074157715

In [46]:
results = trainer.predict(ta_lib.tokenized_billsum_test)

In [47]:
decoded_prediction = ta_lib.tokenizer.batch_decode(results[0], skip_special_tokens=True)

In [48]:
TALib.dump_to_kaggle_format(decoded_prediction , 'pruned_model_0.26_day_5_6.csv')

,ID,Predict
0,0,Amends the Water Resources Development Act of ...
1,1,Federal Forage Fee Act of 1993 - Amends the Fe...
2,2,Merchant Marine of World War II Congressional ...
3,3,Small Business Tax Modernization Act of 2004 -...
4,4,Fair Access to Investment Research Act of 2016...
...,...,...
3264,3264,Public Servant Priority Placement Act of 1995 ...
3265,3265,Sportsmanship of 2008 - Amends the Federal cri...
3266,3266,Helping College Student Cross the Finish Line ...
3267,3267,Texas National Forests Improvement Act of 2000...


In [49]:
TALib.run_score(decoded_prediction , ta_lib.billsum_test)

0.16520012803386583

In [50]:
TALib.save_model(model , "output/pruning_success_v2")

In [51]:
reload_model = TALib.load_model("output/pruning_success_v2")

Some weights of the model checkpoint at output/pruning_success_v2 were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.0.SelfAttention.k.weight_mask', 'decoder.block.0.layer.0.SelfAttention.k.weight_orig', 'decoder.block.0.layer.0.SelfAttention.o.weight_mask', 'decoder.block.0.layer.0.SelfAttention.o.weight_orig', 'decoder.block.0.layer.0.SelfAttention.q.weight_mask', 'decoder.block.0.layer.0.SelfAttention.q.weight_orig', 'decoder.block.0.layer.0.SelfAttention.v.weight_mask', 'decoder.block.0.layer.0.SelfAttention.v.weight_orig', 'decoder.block.0.layer.1.EncDecAttention.k.weight_mask', 'decoder.block.0.layer.1.EncDecAttention.k.weight_orig', 'decoder.block.0.layer.1.EncDecAttention.o.weight_mask', 'decoder.block.0.layer.1.EncDecAttention.o.weight_orig', 'decoder.block.0.layer.1.EncDecAttention.q.weight_mask', 'decoder.block.0.layer.1.EncDecAttention.q.weight_orig', 'decoder.block.0.layer.1.EncDecAttention.v.weight_mask', 'decoder.block.0.layer.1.EncDecAtt

In [52]:
TALib.show_param_ratio(reload_model)

0.26261115074157715

In [53]:
trainer = ta_lib.get_trainer(reload_model , num_train_epochs=30 , batch_size=5 ,output_dir="./output/pruning_check")

/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [54]:
trainer.evaluate(ta_lib.tokenized_billsum_test)

/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'eval_loss': 2.7338263988494873,
 'eval_rouge1': 0.2343,
 'eval_rouge2': 0.1838,
 'eval_rougeL': 0.2253,
 'eval_rougeLsum': 0.2253,
 'eval_gen_len': 18.9838,
 'eval_runtime': 431.505,
 'eval_samples_per_second': 7.576,
 'eval_steps_per_second': 1.516}